In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls /content/gdrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
MyDrive


In [ ]:
!curl ipecho.net/plain

35.229.57.177

In [ ]:
%cd /content/gdrive/My\ Drive/Thesis/MovieLens20M

/content/gdrive/My Drive/Thesis/MovieLens20M


In [ ]:
import glob
import os
import pandas as pd
import numpy as np

In [ ]:
def load_name(csv_path,id):
    df= pd.read_csv(csv_path)
    return (df.loc[ df['movieId'] == id])['title'].values.tolist()
def load_label(csv_path,id):
    df= pd.read_csv(csv_path)
    genres_str = (df.loc[ df['movieId'] == id])['genres'].values.tolist()
    genres = genres_str[0].split('|')
    return genres
def load_all_labels(csv_path):
    download_path = "Data/"
    df= pd.read_csv(csv_path)
    ids =  df['movieId']
    all_labels = []
    for id in ids:
        i = id
        #print(i)
        labels = load_label(csv_path,i)
        all_labels.extend([l for l in labels if l not in all_labels])
        #print(load_name(csv_path,i))
    return all_labels

In [ ]:
def find_exists(csv_path,csv_path2,LABELS):
    download_path = "Data/"

    df= pd.read_csv(csv_path)
    movie_ids = df['movieId']
    for i in movie_ids:
        
        #print(i)
        labels = load_label(csv_path,i)
        a=[l for l in labels if l not in LABELS]
        #print(id, ", ",labels)
        if len(a)> 0 and len(a) == len(labels):
            print("We have a problem houston.", i, ", ",labels)
            print("********")
            print(df.loc[movie_ids == i])
            df.drop(df.loc[df['movieId']==i].index, inplace=True)
            print("-")
            print(df.loc[movie_ids == i])
            
        elif len(a)>0: 
            b = [l for l in labels if l in LABELS]
            new_genre = '|'.join(map(str, b))
            print("********")
            print(df.loc[movie_ids == i])
            df.at[movie_ids == i,['genres']] = new_genre
            print("-")
            print(df.loc[movie_ids == i])

    df.to_csv(csv_path2,index=False)

def merge_labels(label1,label2,csv_in,csv_out):
    download_path = "Data/"
 
    df= pd.read_csv(csv_in)
    movie_ids = df['movieId']
    for i in movie_ids:
        
        #print(i)
        labels = load_label(csv_in,i)
       
        if label1 in labels or label2 in labels:
            b = [l for l in labels if not (l == label1 or l==label2)]
            new_label_name = label1+"-"+label2
            b.append(new_label_name)
            new_genre = '|'.join(map(str, b))
            df.at[movie_ids == i,['genres']] = new_genre
    df.to_csv(csv_out,index=False)

def replace_labels(label_to_replace,new_label,csv_in,csv_out):
    download_path = "Data/"
 
    df= pd.read_csv(csv_in)
    movie_ids = df['movieId']
    for i in movie_ids:
        
        #print(i)
        labels = load_label(csv_in,i)
       
        if label_to_replace in labels:
            b = [l for l in labels if not (l == label_to_replace or l==new_label)]
            b.append(new_label)
            new_genre = '|'.join(map(str, b))
            df.at[movie_ids == i,['genres']] = new_genre
    df.to_csv(csv_out,index=False)

In [ ]:
def find_coocuring(label1,label2,csv_in):
    coocur = 0
    df = pd.read_csv(csv_in)
    movie_ids = df['movieId']
    for i in movie_ids:
        
        #print(i)
        labels = load_label(csv_in,i)
        if label1 in labels and label2 in labels:
            coocur =+ 1
    return coocur
def find_coocuring_matrix(csv_in,size):
    matrix = np.zeros((len(size),len(size)),dtype=int)

    df = pd.read_csv(csv_in)
    movie_ids = df['movieId']
    for id in movie_ids:
        labels = (df.loc[ df['movieId'] == id])['genres'].values.tolist()[0].split('|')
        
        if len(labels)>1:
          for label in labels:
              b = [l for l in labels if l is not label]
              
              for label2 in b:
                  matrix[size.index(label2),size.index(label)] += 1
    return matrix

In [ ]:
def find_all_count(csv_in, all_labels):
    matrix = np.zeros(len(all_labels),dtype=int)

    df = pd.read_csv(csv_in)
    movie_ids = df['movieId']
    for id in movie_ids:
        labels = (df.loc[ df['movieId'] == id])['genres'].values.tolist()[0].split('|')
        for label in labels:
            matrix[all_labels.index(label)] +=1 
    return matrix

In [ ]:
def print_pretty(matrix):
    s = [[str(e) for e in row] for row in matrix]
    lens = [max(map(len, col)) for col in zip(*s)]
    fmt = '\t'.join('{{:{}}}'.format(x) for x in lens)
    table = [fmt.format(*row) for row in s]
    print ('\n'.join(table))

In [ ]:
csv_path = "movies_filtered_2.csv"
LABELS = load_all_labels(csv_path)
print("I have found the following labels. " , LABELS)
print("with length of %d", len(LABELS))
print("now running find coocuring matrix...")



I have found the following labels.  ['Adventure', 'Comedy', 'Fantasy', 'Animation-Children', 'Romance', 'Drama', 'Action', 'Crime', 'Thriller', 'Horror', 'Mystery', 'Sci-Fi', 'Documentary', 'Musical']
with length of %d 14
now running find coocuring matrix...


In [ ]:
#matrix=find_coocuring_matrix("movies_filtered.csv",LABELS)
#print_pretty(matrix)

In [ ]:
matrix2 = find_all_count("movies_filtered_2.csv",LABELS)
print(matrix2)

In [ ]:
#replace_labels("Western","Action","movies_filtered.csv","movies_filtered_2.csv")

In [ ]:
#csv_path = "movies_filtered_2.csv"
#LABELS = load_all_labels(csv_path)
#print("I have found the following labels. " , LABELS)
#print("with lengthu of %d", len(LABELS))
#print("now running find coocuring matrix...")
#print(LABELS)
#matrix=find_coocuring_matrix("movies_filtered_2.csv",LABELS)
#print_pretty(matrix)

#matrix2 = find_all_count("movies_filtered_2.csv",LABELS)
#print(matrix2)

In [ ]:
#merge_labels("Animation","Children","movies_filtered_2.csv","movies_filtered_2.csv")

In [ ]:
#replace_labels("Animation", "Children","movies_filtered_2.csv","movies_filtered_2.csv")

In [ ]:
LABELS = load_all_labels("movies_filtered_2.csv")
print(LABELS)
matrix=find_coocuring_matrix("movies_filtered_2.csv",LABELS)
print_pretty(matrix)

matrix2 = find_all_count("movies_filtered_2.csv",LABELS)
print(matrix2)

In [ ]:
print(LABELS)